In [2]:
import requests
import pandas
import json
import xml.etree.ElementTree as ET
import base64

entw_auth_url = "https://sws-test.verrechnungsstelle.at/api/v1/authenticate"
entw_grundbuch_url = "https://sws-test.verrechnungsstelle.at/api/v1/gb/xml"
entw_firmen_url = "https://sws-test.verrechnungsstelle.at/api/v1/fb/wertetabellen"


auth_login_json = {"username" : "oca-999100", "password" : "7cf71c0df47212d6381a26ad53a5278b"}
api_key = "ACH-T27RCD94UP5RHMAPT27HMCNUTDCUPCPQ5QN5CNE5R4CHZ27DQDSV7VZYS7W3FAX4"

In [3]:
header_post_auth = {'X-API-KEY': api_key}
return_json = requests.post(headers=header_post_auth,url=entw_auth_url, json=auth_login_json)
access_token = return_json.json()['accessToken']

In [4]:
request_json = {

    "table": "FBPLZ",
    "stichtag": "2019-08-24",
    "uvstInfo": 

    {
        "softwareName": "string",
        "softwareVersion": "string",
        "usewareKosten": 0,
        "usewareProdukt": "string",
        "betriebssystem": "string",
        "geraeteName": "string",
        "weitereInfo": "string"
    }

}
header_post_request = {'X-API-KEY': api_key, "Authorization": "Bearer " + access_token}
return_json = requests.post(headers=header_post_request,url=entw_firmen_url, json=request_json)
hashed_xml = return_json.json()['response']

In [20]:
unhashed_xml = base64.b64decode(hashed_xml)
#unhashed_xml = str(unhashed_xml).replace("ns16:", "")

In [64]:
tree = ET.ElementTree(ET.fromstring(unhashed_xml))
tree.write("output.xml")
root = tree.getroot()
is_austria = False

postal_codes = list()

for child in root.iter():
    childtag = child.tag.replace(r"{ns://firmenbuch.justiz.gv.at/Abfrage/v1/FBWertetabellen}","")
    
    if childtag == "STAAT" and child.text == "AUT":
        is_austria = True
    if childtag == "STAAT" and child.text != "AUT":
        is_austria = False
    
    if childtag == "PLZ" and is_austria == True:
        postal_codes.append(child.text)
        
postal_codes

['0000',
 '1000',
 '1004',
 '1006',
 '1010',
 '1011',
 '1012',
 '1013',
 '1014',
 '1015',
 '1016',
 '1017',
 '1018',
 '1020',
 '1021',
 '1022',
 '1023',
 '1024',
 '1025',
 '1026',
 '1027',
 '1029',
 '1030',
 '1031',
 '1032',
 '1033',
 '1034',
 '1035',
 '1036',
 '1037',
 '1038',
 '1039',
 '1040',
 '1041',
 '1042',
 '1043',
 '1044',
 '1045',
 '1046',
 '1047',
 '1050',
 '1051',
 '1052',
 '1053',
 '1060',
 '1061',
 '1062',
 '1063',
 '1064',
 '1065',
 '1070',
 '1071',
 '1072',
 '1075',
 '1080',
 '1081',
 '1082',
 '1090',
 '1091',
 '1092',
 '1093',
 '1094',
 '1095',
 '1096',
 '1097',
 '1100',
 '1101',
 '1102',
 '1103',
 '1104',
 '1105',
 '1106',
 '1107',
 '1108',
 '1109',
 '1110',
 '1111',
 '1112',
 '1113',
 '1114',
 '1115',
 '1120',
 '1121',
 '1122',
 '1123',
 '1124',
 '1125',
 '1126',
 '1127',
 '1128',
 '1130',
 '1131',
 '1132',
 '1133',
 '1134',
 '1136',
 '1140',
 '1141',
 '1142',
 '1143',
 '1144',
 '1145',
 '1146',
 '1147',
 '1148',
 '1150',
 '1151',
 '1152',
 '1153',
 '1155',
 '1156',
 

In [66]:
import pgeocode

nomi = pgeocode.Nominatim('at')
data = nomi.query_postal_code(postal_codes)
data = data[["postal_code", "latitude", "longitude"]]
data = data.dropna()
data.to_csv("Zip_to_coord.csv",sep=";")
data.dropna()

,postal_code,latitude,longitude
4,1010,48.2077,16.370500
13,1020,48.2167,16.400000
22,1030,48.1981,16.394800
32,1040,48.1920,16.367100
40,1050,48.1865,16.354900
...,...,...,...
2722,9974,47.0167,12.371400
2723,9981,46.9378,12.628682
2724,9990,46.8333,12.812600
2725,9991,46.8150,12.843300
